# 1. Etração dos dados

### imports

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

### Criando conexão os bancos de dados (transacional e DW)

In [3]:
DB_HOST_ORIGEM = "localhost"
DB_NAME_ORIGEM = "crm_transacional_db"
DB_USER_ORIGEM = "crm_user"
DB_PASSWORD_ORIGEM = "crm_password"
DB_PORT_ORIGEM = "5432"

conexao_origem_str = (
    f"postgresql://{DB_USER_ORIGEM}:{DB_PASSWORD_ORIGEM}@"
    f"{DB_HOST_ORIGEM}:{DB_PORT_ORIGEM}/{DB_NAME_ORIGEM}"
)
engine_origem = create_engine(conexao_origem_str)


DB_HOST_DW = "localhost"
DB_NAME_DW = "dw_db"
DB_USER_DW = "dw_user"
DB_PASSWORD_DW = "dw_password"
DB_PORT_DW = "5433"

conexao_dw_str = (
    f"postgresql://{DB_USER_DW}:{DB_PASSWORD_DW}@"
    f"{DB_HOST_DW}:{DB_PORT_DW}/{DB_NAME_DW}"
)
engine_dw = create_engine(conexao_dw_str)

### Consumindo dados

In [4]:
query_clientes = "SELECT * FROM CLIENTE"
query_categoria_clientes = "SELECT * FROM CATEGORIA_CLIENTE"
query_categoria_produto = "SELECT * FROM CATEGORIA_PRODUTO"
query_fornecedores = "SELECT * FROM FORNECEDORES"
query_item_vendas = "SELECT * FROM ITEM_VENDAS"
query_localidade = "SELECT * FROM LOCALIDADE"
query_lojas = "SELECT * FROM LOJAS"
query_produtos = "SELECT * FROM PRODUTO"
query_produto_fornecedor = "SELECT * FROM PRODUTO_FORNECEDOR"
query_promocoes = "SELECT * FROM PROMOCOES"
query_vendas = "SELECT * FROM VENDAS"
query_vendedor = "SELECT * FROM VENDEDOR"


df_clientes = pd.read_sql(query_clientes, engine_origem)
df_categoria_clientes = pd.read_sql(query_categoria_clientes, engine_origem)
df_categoria_produto = pd.read_sql(query_categoria_produto, engine_origem)
df_fornecedores = pd.read_sql(query_fornecedores, engine_origem)
df_item_vendas = pd.read_sql(query_item_vendas, engine_origem)
df_localidade = pd.read_sql(query_localidade, engine_origem)
df_lojas = pd.read_sql(query_lojas, engine_origem)
df_produtos = pd.read_sql(query_produtos, engine_origem)
df_produto_fornecedor = pd.read_sql(query_produto_fornecedor, engine_origem)
df_promocoes = pd.read_sql(query_promocoes, engine_origem)
df_vendas = pd.read_sql(query_vendas, engine_origem)
df_vendedor = pd.read_sql(query_vendedor, engine_origem)

### Verificando nome das colunas

In [5]:
dataframes = {
        "df_clientes": df_clientes,
        "df_categoria_clientes": df_categoria_clientes,
        "df_categoria_produto": df_categoria_produto,
        "df_fornecedores": df_fornecedores,
        "df_item_vendas": df_item_vendas,
        "df_localidade": df_localidade,
        "df_lojas": df_lojas,
        "df_produtos": df_produtos,
        "df_produto_fornecedor": df_produto_fornecedor,
        "df_promocoes": df_promocoes,
        "df_vendas": df_vendas,
        "df_vendedor": df_vendedor,
}

for df_name, df in dataframes.items():
        print(f"--- DataFrame: {df_name} ---")
        print("\nColunas:")
        print(df.columns.tolist())
        print("\n" + "="*50 + "\n")

--- DataFrame: df_clientes ---

Colunas:
['id_cliente', 'nome_cliente', 'idade', 'genero', 'id_categoria_cliente', 'id_localidade']


--- DataFrame: df_categoria_clientes ---

Colunas:
['id_categoria_cliente', 'nome_categoria_cliente']


--- DataFrame: df_categoria_produto ---

Colunas:
['id_categoria_produto', 'nome_categoria_produto']


--- DataFrame: df_fornecedores ---

Colunas:
['id_fornecedor', 'nome_fornecedor', 'pais_origem']


--- DataFrame: df_item_vendas ---

Colunas:
['id_venda', 'id_produto', 'qtd_vendida', 'preco_venda', 'id_promocao_aplicada']


--- DataFrame: df_localidade ---

Colunas:
['id_localidade', 'cidade', 'estado', 'regiao']


--- DataFrame: df_lojas ---

Colunas:
['id_loja', 'nome_loja', 'gerente_loja', 'cidade', 'estado']


--- DataFrame: df_produtos ---

Colunas:
['id_produto', 'nome_produto', 'id_categoria_produto']


--- DataFrame: df_produto_fornecedor ---

Colunas:
['id_produto', 'id_fornecedor', 'custo_compra_unitario']


--- DataFrame: df_promocoes ---

# 2. Transformação dos dados

### Dimensão clientes

In [7]:
df_clientes

,id_cliente,nome_cliente,idade,genero,id_categoria_cliente,id_localidade
0,1,Ana Sophia Silveira,48.0,F,3,105
1,2,Dr. Davi Miguel Almeida,63.0,None,4,105
2,3,Sr. Luiz Otávio Farias,28.0,Masculino,1,105
3,4,Ana Beatriz Monteiro,58.0,Feminino,10,105
4,5,Lorenzo Duarte,23.0,F,10,105
...,...,...,...,...,...,...
7995,7996,Sra. Hellena Ferreira,58.0,Outro,10,66
7996,7997,Mariah Souza,51.0,F,2,52
7997,7998,Antônio Peixoto,56.0,Masculino,5,89
7998,7999,Isis Rocha,72.0,F,10,105


In [8]:
# Idades que são nulas, menores que 1 e maiores que 100 recebem a média das idades como valor. Idade passa a ser inteiro.

idade_para_media = df_clientes['idade'].copy()

idade_para_media[idade_para_media < 1] = np.nan
idade_para_media[idade_para_media > 100] = np.nan
media_idades_validas = idade_para_media.mean()
media_idades_validas = round(media_idades_validas)

condicao_idade_invalida = (df_clientes['idade'] < 1) | \
                          (df_clientes['idade'] > 100) | \
                          (df_clientes['idade'].isnull())

df_clientes.loc[condicao_idade_invalida, 'idade'] = media_idades_validas
df_clientes['idade'] = df_clientes['idade'].astype(int)


### Dimensões Fornecedores

In [18]:
df_fornecedores

,id_fornecedor,nome_fornecedor,pais_origem
0,1,Araújo - BRASIL,BRASIL
1,2,Pastor - BRASIL - BRASIL - BRASIL,BRASIL
2,3,Novais - ESTADOS UNIDOS,ESTADOS UNIDOS
3,4,Jesus - CHINA,CHINA
4,5,Souza - BRASIL - BRASIL - BRASIL,BRASIL
...,...,...,...
195,196,Montenegro,BRASIL
196,197,Freitas S/A,CHINA
197,198,Peixoto Ltda.,CHINA
198,199,Oliveira,PAIS NÃO INFORMADO


In [7]:
# Usa a condição para localizar as linhas com país nulo e atribuir o valor "PAIS NÃO INFORMADO".

valor_substituto = "PAIS NÃO INFORMADO"

condicao_pais_nulo = df_fornecedores['pais_origem'].isnull()

df_fornecedores.loc[condicao_pais_nulo, 'pais_origem'] = valor_substituto

In [10]:
# Get nos nomes dos paises

paises_limpos = df_fornecedores['pais_origem'].str.upper().str.strip()

paises_distintos = paises_limpos.unique()

print("Os seguintes valores distintos foram encontrados na coluna 'pais_origem':")
print(paises_distintos)

Os seguintes valores distintos foram encontrados na coluna 'pais_origem':
['BRASIL' 'ESTADOS UNIDOS' 'CHINA' 'PAIS NÃO INFORMADO']


In [9]:
# Padronização dos nomes dos paises

df_fornecedores['pais_origem'] = df_fornecedores['pais_origem'].str.upper().str.strip()

mapeamento_paises = {
    'BR': 'BRASIL',
    'BRAZIL': 'BRASIL',
    'CN': 'CHINA',
    'USA': 'ESTADOS UNIDOS',
    'EUA': 'ESTADOS UNIDOS'
}

df_fornecedores['pais_origem'] = df_fornecedores['pais_origem'].replace(mapeamento_paises)

In [17]:
contagem_nomes = df_fornecedores['nome_fornecedor'].value_counts()
nomes_duplicados = contagem_nomes[contagem_nomes > 1].index.tolist()

if nomes_duplicados:
    condicao_duplicados = df_fornecedores['nome_fornecedor'].isin(nomes_duplicados)

    df_fornecedores.loc[condicao_duplicados, 'nome_fornecedor'] = \
        df_fornecedores.loc[condicao_duplicados, 'nome_fornecedor'] + ' - ' + df_fornecedores.loc[condicao_duplicados, 'pais_origem']
else:
    print("Não foram encontrados nomes de fornecedor duplicados para tratar.")

df_fornecedores.drop_duplicates(subset=['nome_fornecedor'], keep='first', inplace=True)


### Dimensões Vendedor

In [14]:
df_vendedor

,id_vendedor,nome_vendedor
0,1,Lívia da Rocha
1,2,Levi Borges
2,3,Ana Laura Fernandes
3,4,Luiz Fernando Vargas
4,5,Davi Lucas Pereira
...,...,...
495,496,Sr. João Miguel Costela
496,497,Antony Nascimento
497,498,Bryan Ferreira
498,499,Pedro Henrique Santos


In [15]:
# Remoção de prefixos nos nomes
import re

prefixos_a_remover = ['SR\.', 'SRA\.', 'DR\.', 'DRA\.', 'Srta\.']
padrao_regex = r'^(?:' + '|'.join(prefixos_a_remover) + r')\s*'

df_vendedor['nome_vendedor'] = df_vendedor['nome_vendedor'].str.replace(
    padrao_regex,
    '',
    regex=True,
    flags=re.IGNORECASE
).str.strip()

In [16]:
# Removendo nomes duplicados

df_vendedor.drop_duplicates(subset=['nome_vendedor'], keep='first', inplace=True)

### Dimensões Promoções

In [17]:
df_promocoes

,id_promocao,nome_promocao,tipo_desconto,data_inicio,data_fim
0,1,Promoção Soluta,Fixo,2024-12-30,Data Inválida
1,2,Promoção Veniam,None,2025-01-29,2025-03-15
2,3,Promoção Assumenda,Valor Fixo,2024-06-05,2024-07-23
3,4,Promoção Dolor,%,2022-07-18,2022-08-02
4,5,Promoção Doloribus,None,2024-06-08,2024-07-31
5,6,Promoção Qui,None,N/A,2025-06-11
6,7,Promoção Eveniet,None,2024-12-30,2025-01-23
7,8,Promoção Eaque,Valor Fixo,2022-10-12,2022-11-02
8,9,Promoção Pariatur,None,N/A,Data Inválida
9,10,Promoção Possimus,None,03/10/2024,2024-10-20


In [18]:
# Preenche os valores nulos em tipo_desconto com "Não informado"

df_promocoes['tipo_desconto'] = df_promocoes['tipo_desconto'].fillna('NAO INFORMADO')

In [19]:
# Unifica os sinônimos e símbolos para um padrão

df_promocoes['tipo_desconto'] = df_promocoes['tipo_desconto'].str.upper().str.strip()

mapeamento_desconto = {
    'FIXO': 'VALOR FIXO',
    '%': 'PERCENTUAL'
}
df_promocoes['tipo_desconto'] = df_promocoes['tipo_desconto'].replace(mapeamento_desconto)

In [20]:
# Corrigir inconsistencias na coluna "data_fim".

df_promocoes['data_inicio'] = df_promocoes['data_inicio'].replace(['N/A', 'Data Inválida', ''], np.nan)

df_promocoes['data_inicio'] = pd.to_datetime(df_promocoes['data_inicio'], errors='coerce')

In [21]:
# Corrigir inconsistencias na coluna "data_fim".

df_promocoes['data_fim'] = df_promocoes['data_fim'].replace(['Data Inválida', ''], np.nan)

### Dimensões loja

In [22]:
df_lojas

,id_loja,nome_loja,gerente_loja,cidade,estado
0,1,Loja Brito do Campo,João Miguel Duarte,Sousa,PA
1,2,Loja Guerra Verde,Davi Miguel Pastor,Oliveira,RO
2,3,Loja Souza,Sr. José Pedro Duarte,Correia,MT
3,4,Loja Cunha,Liz Melo,Carvalho do Norte,PI
4,5,Loja Machado,Maria Castro,Fonseca,PI
5,6,Loja Vasconcelos do Norte,Aylla Viana,Fonseca,PA
6,7,Loja Monteiro do Amparo,Igor da Rosa,,BA
7,8,Loja das Neves,Marcela Vasconcelos,Rios,AC
8,9,Loja Fernandes,Bianca Farias,Porto,MG
9,10,Loja Caldeira,Caio Silva,Dias de Nascimento,AC


In [23]:
# Resolvendo as inconsistencias de cidade, rem

import numpy as np

df_lojas['cidade'] = df_lojas['cidade'].replace('', np.nan)

df_lojas['cidade'] = df_lojas['cidade'].fillna('CIDADE NÃO INFORMADA')


In [24]:
# Unindo o nome das lojas com nomes iguais com o nome de suas cidades.

contagem_nomes = df_lojas['nome_loja'].value_counts()
nomes_duplicados = contagem_nomes[contagem_nomes > 1].index.tolist()

condicao_duplicados = df_lojas['nome_loja'].isin(nomes_duplicados)

df_lojas.loc[condicao_duplicados, 'nome_loja'] = \
    df_lojas.loc[condicao_duplicados, 'nome_loja'] + ' - ' + df_lojas.loc[condicao_duplicados, 'cidade']

In [25]:
# Remove prefixos dos nomes dos gerentes

import re

prefixos_a_remover = ['SR\.', 'SRA\.', 'DR\.', 'DRA\.', 'Srta\.']
padrao_regex = r'^(?:' + '|'.join(prefixos_a_remover) + r')\s*'
df_lojas['gerente_loja'] = df_lojas['gerente_loja'].str.replace(
    padrao_regex,
    '',
    regex=True,
    flags=re.IGNORECASE
).str.strip()